In [3]:
from dataclasses import dataclass, field
from typing import Tuple, Iterator
from parse import parse
from collections import Counter, defaultdict
from math import prod
import string
import hashlib
import re
import numpy as np

import black
import jupyter_black


jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [32]:
# Day 1: No Time for a Taxicab
@dataclass
class Position:
    heading: int
    x: int
    y: int
    visited: set = field(default_factory=set)
    double_visit_distance: int = 0

    # NESW = 0, 1, 2, 3
    HEADINGS = ((0, 1), (1, 0), (0, -1), (-1, 0))

    def forward(self, steps: int):
        "Move forward `steps` given current heading"
        for _ in range(steps):
            self.x = self.x + self.HEADINGS[self.heading][0]
            self.y = self.y + self.HEADINGS[self.heading][1]

            if not self.double_visit_distance:
                if (self.x, self.y) in self.visited:
                    self.double_visit_distance = abs(self.x) + abs(self.y)
                self.visited.add((self.x, self.y))

    def turn_right(self):
        self.heading = (self.heading + 1) % 4

    def turn_left(self):
        self.heading = (self.heading - 1) % 4


directions = [(dir[0], int(dir[1:])) for dir in open("2016/1.txt").read().split(", ")]

pos = Position(0, 0, 0)
for turn, steps in directions:
    if turn == "R":
        pos.turn_right()
    elif turn == "L":
        pos.turn_left()
    else:
        raise ValueError(dir)
    pos.forward(steps)

print(f"Part 1: {abs(pos.x) + abs(pos.y)}")  # 236
print(f"Part 2: {pos.double_visit_distance}")  # 182

Part 1: 236
Part 2: 182


In [21]:
# Day 2: Bathroom Security
@dataclass
class Keypad:
    """
    Implements a keypad where you can move up, down, right, and left. You can't move past
    an edge. Represents current value as a string of this object.

    Keypad looks like this:

    1 2 3
    4 5 6
    7 8 9
    """

    number: int = 5

    def up(self):
        if self.number not in (1, 2, 3):
            self.number -= 3

    def down(self):
        if self.number not in (7, 8, 9):
            self.number += 3

    def left(self):
        if self.number not in (1, 4, 7):
            self.number -= 1

    def right(self):
        if self.number not in (3, 6, 9):
            self.number += 1

    def __str__(self):
        return str(self.number)


@dataclass
class BathroomKeypad:
    """
    Keypad looks like below.

        1
      2 3 4
    5 6 7 8 9
      A B C
        D
    """

    number: int = 5
    # Internal representation of keypad
    #       1
    #    2  3  4
    # 5  6  7  8  9
    #   10 11 12
    #      13

    def up(self):
        if self.number not in (5, 2, 1, 4, 9):
            if self.number in (3, 13):
                self.number -= 2
            elif self.number in (6, 7, 8, 10, 11, 12):
                self.number -= 4

    def down(self):
        if self.number not in (5, 10, 13, 12, 9):
            if self.number in (1, 11):
                self.number += 2
            else:
                self.number += 4

    def left(self):
        if self.number not in (1, 2, 5, 10, 13):
            self.number -= 1

    def right(self):
        if self.number not in (1, 4, 9, 12, 13):
            self.number += 1

    def __str__(self) -> str:
        return hex(self.number)[-1]


lines = open("2016/2.txt").readlines()

a = b = ""
keypad = Keypad()
bathroom = BathroomKeypad()
for line in lines:
    for move in line:
        match move:
            case "U":
                keypad.up()
                bathroom.up()
            case "D":
                keypad.down()
                bathroom.down()
            case "L":
                keypad.left()
                bathroom.left()
            case "R":
                keypad.right()
                bathroom.right()
    a += str(keypad)
    b += str(bathroom)
print(f"Part 1: {a}")
print(f"Part 1: {b}")

Part 1: 52981
Part 1: 74cd2


In [220]:
# Day 3: Squares With Three Sides
def possible(sides: Tuple[int, int, int]) -> bool:
    "Returns true if this combination is possible."
    a, b, c = sorted(sides)
    return a + b > c


lines = open("2016/3.txt").readlines()
lines = [line.strip().split() for line in lines]

sides = [tuple(int(side) for side in line) for line in lines]
print(f"Part 1: {sum(possible(side) for side in sides)}")  # 869

sides = [side for i in range(0, len(sides), 3) for side in zip(*sides[i : i + 3])]
print(f"Part 2: {sum(possible(side) for side in sides)}")  # 1544

Part 1: 869
Part 2: 1544


In [238]:
# Day 4: Security Through Obscurity
def real_room(name: str, checksum: str) -> bool:
    cnt = Counter(sorted(name.replace("-", "")))
    chk = "".join([kv[0] for kv in cnt.most_common(5)])
    return chk == checksum


def decrypt_room(room: str, steps: int) -> str:
    az = string.ascii_lowercase
    shift = steps % len(az)
    tr = str.maketrans(az, az[shift:] + az[:shift])
    return room.translate(tr)


lines = open("2016/4.txt").readlines()
rooms = [parse("{}-{:d}[{}]", line.strip()) for line in lines]
id_sum = 0
for name, sector, checksum in rooms:
    if real_room(name, checksum):
        id_sum += sector
        # Part 2: Sector ID of North Pole objects
        if "north" in decrypt_room(name, sector):
            print(f"Part 2: {sector}: {decrypt_room(name, sector)}")  # 482
print(f"Part 1: {id_sum}")

Part 2: 482: northpole-object-storage
Part 1: 361724


In [2]:
# Day 5: How About a Nice Game of Chess
# 10 s execution time part 1, 40 s when including part 2
base = "abbhdwsy"
pass1 = []
pass2 = [0] * 8
index = 0
while not all(pass2):
    candidate = base + str(index)
    hash = hashlib.md5(candidate.encode("utf-8")).hexdigest()
    if hash.startswith("00000"):
        if len(pass1) < 8:
            pass1.append(hash[5])
        pos = int(hash[5], base=16)
        if pos < 8 and not pass2[pos]:
            pass2[pos] = hash[6]
    index += 1
print(f'Part 1: {"".join(pass1[:8])}')
print(f'Part 2: {"".join(pass2)}')

Part 1: 801b56a7
Part 2: 424a0197


In [97]:
# Day 6: Signals and Noise
lines = open("2016/6.txt").read()
messages = [line.strip() for line in lines.splitlines()]
counters = [Counter(column) for column in (zip(*messages))]
print(f'Part 1: {"".join(c.most_common(1)[0][0] for c in counters)}') # tsreykjj
print(f'Part 2: {"".join(c.most_common()[-1][0][0] for c in counters)}') # hnfbujie

Part 1: tsreykjj
Part 2: hnfbujie


In [98]:
# Day 7: Internet Protocol Version 7
def abba(text: str, pos: int) -> bool:
    if text[pos] == text[pos + 3] and text[pos + 1] == text[pos + 2]:
        if text[pos] != text[pos + 1]:
            return True


def supports_tls(text: str) -> bool:
    abba_outside = False
    outside_brackets = True
    for i in range(len(text) - 3):
        if text[i] == "[":
            outside_brackets = False
            next
        if text[i] == "]":
            outside_brackets = True
            next
        if abba(text, i):
            if outside_brackets:
                abba_outside = True
            else:
                return False
    return abba_outside


assert supports_tls("abba[mnop]qrst") == True
assert supports_tls("abcd[bddb]xyyx") == False
assert supports_tls("aaaa[qwer]tyui") == False
assert supports_tls("ioxxoj[asdfgh]zxcvbn") == True


def aba_outside(text: str) -> str:
    supernet = True
    for a, b, c in zip(text, text[1:], text[2:]):
        if a == "[":
            supernet = False
            next
        if a == "]":
            supernet = True
            next
        if supernet and a == c and a != b:
            yield a + b + a


def bab_inside(text: str, candidate: str) -> bool:
    hypernet = False
    for a, b, c in zip(text, text[1:], text[2:]):
        if a == "[":
            hypernet = True
            next
        if a == "]":
            hypernet = False
            next
        if hypernet and a == c and candidate == b + a + b:
            return True


def supports_sls(text: str) -> bool:
    return any(bab_inside(text, candidate) for candidate in aba_outside(text))


assert supports_sls("aba[bab]xyz") == True
assert supports_sls("xyx[xyx]xyx") == False
assert supports_sls("aaa[kek]eke") == True
assert supports_sls("zazbz[bzb]cdb") == True

lines = open("2016/7.txt").read()
lines = [line for line in lines.splitlines()]
print(f"Part 1: {sum(supports_tls(line) for line in lines)}")  # 115
print(f"Part 2: {sum(supports_sls(line) for line in lines)}")  # 231

Part 1: 115
Part 2: 231


In [36]:
# Day 8: Two-Factor Authentication
lines = open("2016/8.txt").read().splitlines()
commands = [[line] + [int(x) for x in re.findall(r"[\d]+", line)] for line in lines]
display = [[0] * 50 for _ in range(6)]


def turn_on(display, wide: int, tall: int):
    for r in range(tall):
        for c in range(wide):
            display[r][c] = 1
    return display


def rotate_row(display, r: int, steps: int):
    "Rotate row `r` to the right"
    row = display[r]
    display[r] = row[len(row) - steps :] + row[: len(row) - steps]
    return display


def rotate_column(display, c: int, steps: int):
    "Rotate column `c` down"
    transposed = list(zip(*display))
    transposed = rotate_row(transposed, c, steps)
    display = list(zip(*transposed))
    return [list(row) for row in display]


def print_display(display):
    for row in range(len(display)):
        for col in range(len(display[row])):
            if display[row][col]:
                print("##", end="")
            else:
                print("  ", end="")
        print()


for command, a, b in commands:
    if command.startswith("rect"):
        display = turn_on(display, wide=a, tall=b)
    if command.startswith("rotate row"):
        display = rotate_row(display, r=a, steps=b)
    if command.startswith("rotate column"):
        display = rotate_column(display, c=a, steps=b)
print(f"Part 1: {sum(val for line in display for val in line)}")
print_display(display)

Part 1: 123
  ####    ########  ######    ##    ##  ######    ########  ######        ####  ######      ######  
##    ##  ##        ##    ##  ##    ##  ##    ##        ##  ##    ##        ##  ##    ##  ##        
##    ##  ######    ######    ##    ##  ##    ##      ##    ######          ##  ##    ##  ##        
########  ##        ##    ##  ##    ##  ######      ##      ##    ##        ##  ######      ####    
##    ##  ##        ##    ##  ##    ##  ##        ##        ##    ##  ##    ##  ##              ##  
##    ##  ##        ######      ####    ##        ########  ######      ####    ##        ######    


In [5]:
# Day 8 with numpy
lines = open("2016/8.txt").read().splitlines()
commands = [[line] + [int(x) for x in re.findall(r"[\d]+", line)] for line in lines]

display = np.zeros((6, 50), dtype=np.int8)
for command, a, b in commands:
    if command.startswith("rect"):
        display[:b, :a] = 1
    if command.startswith("rotate row"):
        display[a] = np.roll(display[a], shift=b)
    if command.startswith("rotate column"):
        display[:, a] = np.roll(display[:, a], shift=b)
        # Can also use:
        # display.T[a] = np.roll(display.T[a], shift=b)
print(f"Part 1: {np.sum(display)}")
for row in display:
    print("".join("⬜️🟦"[point * 2 : point * 2 + 2] for point in row))

Part 1: 123
⬜️🟦🟦⬜️⬜️🟦🟦🟦🟦⬜️🟦🟦🟦⬜️⬜️🟦⬜️⬜️🟦⬜️🟦🟦🟦⬜️⬜️🟦🟦🟦🟦⬜️🟦🟦🟦⬜️⬜️⬜️⬜️🟦🟦⬜️🟦🟦🟦⬜️⬜️⬜️🟦🟦🟦⬜️
🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️
🟦⬜️⬜️🟦⬜️🟦🟦🟦⬜️⬜️🟦🟦🟦⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️⬜️⬜️🟦⬜️⬜️🟦🟦🟦⬜️⬜️⬜️⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️
🟦🟦🟦🟦⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦🟦🟦⬜️⬜️⬜️🟦⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️🟦🟦🟦⬜️⬜️⬜️🟦🟦⬜️⬜️
🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️⬜️⬜️⬜️🟦⬜️
🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️🟦🟦🟦⬜️⬜️⬜️🟦🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦🟦🟦🟦⬜️🟦🟦🟦⬜️⬜️⬜️🟦🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦🟦🟦⬜️⬜️


In [68]:
# Day 9: Explosives in Cyberspace
def text_length(text: str, part_one=False) -> int:
    "Expands first marker if part_one and all markers found otherwise. Returns length after expansion."
    match = re.search(r"\((\d+)x(\d+)\)", text)
    if not match:
        return len(text)
    length = int(match[1])
    repetitions = int(match[2])
    head = match.start()
    middle = text[match.end() : match.end() + length]
    tail = text[match.end() + length :]
    if part_one:
        return head + length * repetitions + text_length(tail, part_one)
    else:
        return head + repetitions * text_length(middle) + text_length(tail)


text = open("2016/9.txt").read().strip()

print(f"Part 1: {text_length(text, part_one=True)}")  # 115118
print(f"Part 2: {text_length(text)}")  # 11107527530

Part 1: 115118
Part 2: 11107527530


In [166]:
# Day 10: Balance Bots
@dataclass
class BotRule:
    low_dest: str = ""
    low: int = -1
    high_dest: str = ""
    high: int = -1
    low_value: int = -1
    high_value: int = -1


def parse_lines(lines):
    for line in lines:
        if p := parse("value {:d} goes to bot {:d}", line):
            value, bot = p
            bots[bot].append(value)
        elif p := parse("bot {:d} gives low to {} {:d} and high to {} {:d}", line):
            bot, *rules = p
            bot_rules[bot] = BotRule(*rules)
        else:
            raise ValueError("Can not parse", line)


def process_bot(bot: int):
    assert len(bots[bot]) == 2
    # Update rules
    bot_rule = bot_rules[bot]
    bot_rule.low_value = min(bots[bot])
    bot_rule.high_value = max(bots[bot])
    # Give away chips
    if bot_rule.low_dest == "bot":
        bots[bot_rule.low].append(bot_rule.low_value)
    elif bot_rule.low_dest == "output":
        outputs[bot_rule.low].append(bot_rule.low_value)
    else:
        raise ValueError("Bad rule", bot_rule.low_dest)
    if bot_rule.high_dest == "bot":
        bots[bot_rule.high].append(bot_rule.high_value)
    elif bot_rule.high_dest == "output":
        outputs[bot_rule.high].append(bot_rule.high_value)
    else:
        raise ValueError("Bad rule", bot_rule.high_dest)

    bots[bot] = []


def bot_to_process() -> int:
    for key, value in bots.items():
        if len(value) > 1:
            return key
    return -1


bots = defaultdict(lambda: list())
bot_rules = {}
outputs = defaultdict(lambda: list())
lines = open("2016/10.txt").read().strip().splitlines()
parse_lines(lines)

while (bot := bot_to_process()) != -1:
    process_bot(bot)

for bot, rule in bot_rules.items():
    if rule.low_value == 17 and rule.high_value == 61:
        print(f"Part 1: {bot}")  # 157
print(f"Part 2: {prod(outputs[i][0] for i in (0, 1, 2))}")  # 1085

Part 1: 157
Part 2: 1085


In [167]:
# Day 11: Radioisotope Thermoelectric Generators - Not solved yet

"""
The first floor contains a promethium generator and a promethium-compatible microchip.
The second floor contains a cobalt generator, a curium generator, a ruthenium generator, and a plutonium generator.
The third floor contains a cobalt-compatible microchip, a curium-compatible microchip, a ruthenium-compatible microchip, and a plutonium-compatible microchip.
The fourth floor contains nothing relevant.
"""

'\nThe first floor contains a promethium generator and a promethium-compatible microchip.\nThe second floor contains a cobalt generator, a curium generator, a ruthenium generator, and a plutonium generator.\nThe third floor contains a cobalt-compatible microchip, a curium-compatible microchip, a ruthenium-compatible microchip, and a plutonium-compatible microchip.\nThe fourth floor contains nothing relevant.\n'

In [1]:
# Day 12: Leonardo's Monorail
def value(x):
    try:
        return int(x)
    except ValueError:
        return reg[x]


pc = 0
reg = {"a": 0, "b": 0, "c": 0, "d": 0}
# reg["c"] = 1 # Uncomment to run part 2, takes 30s to finish
instructions = open("2016/12.txt").read().strip().splitlines()
while pc < len(instructions):
    match instructions[pc].split():
        case "cpy", x, y:
            reg[y] = value(x)
        case "inc", x:
            reg[x] += 1
        case "dec", x:
            reg[x] -= 1
        case "jnz", x, y:
            if value(x) != 0:
                pc = pc + int(y) - 1
    pc += 1
print(f'a: {reg["a"]}')  # 318003

a: 318003


In [54]:
# Day 13: A Maze of Twisty Little Cubicles
from heapq import heappop, heappush
from math import inf


def is_wall(x: int, y: int, fav: int) -> bool:
    num = x * x + 3 * x + 2 * x * y + y + y * y + fav
    return bin(num).count("1") % 2


def in_bounds(x: int, y: int) -> bool:
    return 0 <= x and 0 <= y


def valid_paths(x: int, y: int, fav: int):
    candidates = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
    return [
        c for c in candidates if in_bounds(c[0], c[1]) and not is_wall(c[0], c[1], fav)
    ]


def steps_to(goal_x: int, goal_y: int, fav: int) -> int:
    frontier = []
    heappush(frontier, (0, (1, 1)))  # We start at 1,1
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[(1, 1)] = 0

    while frontier:
        x, y = heappop(frontier)[1]

        if (x, y) == (goal_x, goal_y):
            break

        for (next_x, next_y) in valid_paths(x, y, fav):
            new_cost = cost_so_far[(x, y)] + 1
            if new_cost < cost_so_far[(next_x, next_y)]:
                cost_so_far[(next_x, next_y)] = new_cost
                heappush(frontier, (new_cost, (next_x, next_y)))

    return cost_so_far


locations = steps_to(31, 39, 1362)

print(f"Part 1: {locations[(31,39)]}")  # 82
print(f"Part 2: {sum(1 for dist in locations.values() if dist <= 50)}")

Part 1: 82
Part 2: 138
